In [32]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [33]:
# H6.1(a): Convolutional neural network


x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
    
    
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Train
for _ in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))

0.9242


In [34]:
# H6.1(b): Convolutional neural network

learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_hidden_3 = 256 # 3rd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.truncated_normal(shape = [n_input, n_hidden_1], mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)),
    'h2': tf.Variable(tf.truncated_normal(shape = [n_hidden_1, n_hidden_2], mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)),
    'h3': tf.Variable(tf.truncated_normal(shape = [n_hidden_2, n_hidden_3], mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)),
    'out': tf.Variable(tf.truncated_normal(shape = [n_hidden_3, n_classes],mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None))
}
biases = {
    'b1': tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[n_hidden_1])),
    'b2': tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[n_hidden_2])),
    'b3': tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[n_hidden_3])),
    'out': tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[n_classes]))
}

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 1500 neurons
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    # Hidden fully connected layer with 1500 neurons
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    # Hidden fully connected layer with 1500 neurons
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']))
    # Output fully connected layer with a neuron for each class
    out_layer = tf.nn.relu(tf.matmul(layer_3, weights['out']) + biases['out'])
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-08)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

('Epoch:', '0001', 'cost=207.024674189')
('Epoch:', '0002', 'cost=2.424256023')
('Epoch:', '0003', 'cost=2.306154952')
('Epoch:', '0004', 'cost=2.280731828')
('Epoch:', '0005', 'cost=2.278764169')
('Epoch:', '0006', 'cost=2.280941150')
('Epoch:', '0007', 'cost=2.279685196')
('Epoch:', '0008', 'cost=2.279601472')
('Epoch:', '0009', 'cost=2.279936388')
('Epoch:', '0010', 'cost=2.280145714')
('Epoch:', '0011', 'cost=2.279978253')
('Epoch:', '0012', 'cost=2.279727065')
('Epoch:', '0013', 'cost=2.280103848')
('Epoch:', '0014', 'cost=2.280103851')
('Epoch:', '0015', 'cost=2.279099085')
Optimization Finished!
('Accuracy:', 0.1081)


In [38]:
# H6.1(c): Convolutional neural network
keep_prob = tf.placeholder(tf.float32)
dropout = 0.5
def multilayer_perceptron_dropout(x):
    # Hidden fully connected layer with 1500 neurons
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    # Hidden fully connected layer with 1500 neurons
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    layer_2 = tf.nn.dropout(layer_2, keep_prob)
    # Hidden fully connected layer with 1500 neurons
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']))
    layer_3 = tf.nn.dropout(layer_3, keep_prob)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.nn.relu(tf.matmul(layer_3, weights['out']) + biases['out'])
    return out_layer

# Construct model
logits = multilayer_perceptron_dropout(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-08)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y, keep_prob: 0.5})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels, keep_prob: 0.5}))


('Epoch:', '0001', 'cost=2359.427681919')
('Epoch:', '0002', 'cost=39.982846577')
('Epoch:', '0003', 'cost=14.574386391')
('Epoch:', '0004', 'cost=8.793618303')
('Epoch:', '0005', 'cost=4.880583517')
('Epoch:', '0006', 'cost=5.657732509')
('Epoch:', '0007', 'cost=3.750380350')
('Epoch:', '0008', 'cost=3.563727778')
('Epoch:', '0009', 'cost=2.840681129')
('Epoch:', '0010', 'cost=2.826663709')
('Epoch:', '0011', 'cost=2.536143534')
('Epoch:', '0012', 'cost=2.636626306')
('Epoch:', '0013', 'cost=2.646878228')
('Epoch:', '0014', 'cost=2.587732431')
('Epoch:', '0015', 'cost=2.439223036')
Optimization Finished!
('Accuracy:', 0.097900003)
